<center><img src="https://github.com/DACSS-Spatial/data_forSpatial/raw/main/logo.png" width="700"></center>



<a target="_blank" href="https://colab.research.google.com/github/DACSS-Spatial/binaryOPS/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Binary Spatial operations on  Geo Dataframes

This time the spatial operations will be applied when we have as input two GDFs.

## Getting ready

The links to the our maps on GitHub are here:

In [ ]:
mainLink='https://github.com/DACSS-Spatial/data_forSpatial/raw/main/'

linkWorldMap=mainLink + "WORLD/worldMaps.gpkg"
linkIndicators=mainLink + "WORLD/worldindicators.json"
linkBrazil=mainLink + "BRAZIL/brazil_5880.gpkg"


Let me introduce another [data source on seaports](https://msi.nga.mil/Publications/WPI). Their locations in long/lat is stored in the file **seaports.csv** which we already have on GitHub:

In [ ]:
linkSeaPorts=mainLink + 'WORLD/seaports.csv'

Let's get some maps:

In [ ]:
import geopandas as gpd

#world
world_rivers=gpd.read_file(linkWorldMap,layer='rivers')
#brazil
brazil5880=gpd.read_file(linkBrazil,layer='country')
airports_brazil5880=gpd.read_file(linkBrazil,layer='airports')
states_brazil5880=gpd.read_file(linkBrazil,layer='states')
municipalities_brazil5880=gpd.read_file(linkBrazil,layer='municipalities')
#some indicators
indicators=gpd.read_file(linkIndicators)

# the seaports
import pandas as pd
infoseaports=pd.read_csv(linkSeaPorts)

In [ ]:
# the sesports data has too manny columns:
len(infoseaports.columns)

Let me keep some columns, and turn the DF into a GDF:

In [ ]:
#rename
infoseaports.rename(columns={'Main Port Name':'seaport_name','Country Code':'country_name'},inplace=True)

#keep few columns
infoseaports=infoseaports.loc[:,['seaport_name', 'country_name','Latitude', 'Longitude']]

#spatial points (unprojected)
seaports=gpd.GeoDataFrame(data=infoseaports.copy(),
                           geometry=gpd.points_from_xy(infoseaports.Longitude,
                                                       infoseaports.Latitude),
                          crs=4326)# notice it is unprojected

# keep Brazil
seaports_bra=seaports[seaports['country_name']=='Brazil'].copy()

# reset indexes
seaports_bra.reset_index(drop=True, inplace=True)

# reprojecting
seaports_brazil5880=seaports_bra.to_crs(5880) # projected crs

**Before proceeding**, let me compute some GDFs we created the previous session:


* The centroid of Brazil

In [ ]:
brazil5880_cen=brazil5880.centroid

- Large Brazilian airports

In [ ]:
large_airports=airports_brazil5880.query("airport_type=='large_airport'")

- Amazon Rivers

In [ ]:
AmazonSystem=world_rivers.query("SYSTEM=='Amazon'")
AmazonSystem_5880=AmazonSystem.to_crs(5880)

- The states in the East, West, South, and North

In [ ]:
mid_x,mid_y=brazil5880_cen.x[0],brazil5880_cen.y[0]

# the north
N_brazil=states_brazil5880.cx[:,mid_y:]
# the south
S_brazil=states_brazil5880.cx[:,:mid_y]
# the west
W_brazil=states_brazil5880.cx[:mid_x,:]
# the east
E_brazil=states_brazil5880.cx[mid_x:,:]

- The mean fragility by region of the world

In [ ]:
indicatorsByRegion=indicators.dissolve(
    by="region", #groupby()
    aggfunc={"fragility": "mean"}, #agg()
    )

In [ ]:
# You may need to install if using Colab
# !pip install mapclassify


_____________



## I. Distance

Distance is a key binary operation as so many practical policy matters depend on knowing distances between objects in space.

Any pair of rightly projected GDFs have a distance between them. Below we can make  query using distances:

> Which are the airports whose distance to Brazil centroid is > 2500000?

In [ ]:
# this is the centroid we have:
brazil5880_cen

Then,

In [ ]:
airports_brazil5880[airports_brazil5880.distance(brazil5880_cen[0]) > 2500000]

The results can be confirmed visually:

In [ ]:
base=airports_brazil5880[airports_brazil5880.distance(brazil5880_cen.iloc[0]) > 2500000].plot(marker='+',markersize=100)
airports_brazil5880.plot(ax=base,color='grey', markersize=0.1)
brazil5880_cen.plot(ax=base,color='red')

Let me review how distances work between different kinds of geometries:

### a.  Distance between points

We have these points:


In [ ]:
seaports_brazil5880.head()

In [ ]:
large_airports.head()

For results of distance operation to be more insightful, we may do this:

In [ ]:
# index will be names of airports/seaports instead of numbers
large_airports.set_index('airport_name',inplace=True)
seaports_brazil5880.set_index('seaport_name',inplace=True)

Notice:

In [ ]:
seaports_brazil5880.head()

In [ ]:
large_airports.head()

> The distance from each airport to the "Dtse / Gegua Oil Terminal"

In [ ]:
large_airports.distance(seaports_brazil5880.geometry.loc['Dtse / Gegua Oil Terminal'])

What about computing...

> All the distances between large aiports and seaports (in km)

In [ ]:
# apply creates a LOOP, computes distances from each seaport to all large airports
seaports_brazil5880.geometry.apply\
(lambda seaport: large_airports.geometry.distance(seaport)/1000)

If we save the matrix...

In [ ]:
D_Matrix_sea_air=seaports_brazil5880.geometry.apply \
                (lambda seaport: large_airports.geometry.distance(seaport)/1000)

We can compute some distance stats from there:

> Some summary of distances from each seaport to all large airports

In [ ]:
Stat_sea_air=pd.DataFrame()
Stat_sea_air['mean']=D_Matrix_sea_air.mean(axis=1) # mean D to all airports
Stat_sea_air['min']=D_Matrix_sea_air.min(axis=1)# min D to all airports
Stat_sea_air['max']=D_Matrix_sea_air.max(axis=1)# max D to all airports

# see some
Stat_sea_air.head(10)

Of course, the idmax and idmin could come in handy:
> Farthest airport to each seaport

In [ ]:
D_Matrix_sea_air.idxmax(axis=1).head()

> Farthest seaport to each airport

In [ ]:
D_Matrix_sea_air.idxmax(axis=0).head()

> Closest airport to each seaport

In [ ]:
D_Matrix_sea_air.idxmin(axis=1).head()

> Closest seaport to each airport


In [ ]:
# closest seaport to each airport
D_Matrix_sea_air.idxmin(axis=0).head()

### b. Distance between line and point

Once we know understand how **distance** and idxmin/idxmax work, we can feel comfortable in this stage.

Let's use these rivers from before:

In [ ]:
AmazonSystem_5880.set_index("RIVER",inplace=True)
AmazonSystem_5880

Then,
> Distance from river Tapajos to Guarulhos airport

In [ ]:
airName='Guarulhos - Governador André Franco Montoro International Airport'
rivName='Tapajos'
AmazonSystem_5880.geometry.loc[rivName].distance(large_airports.geometry.loc[airName])/1000

We can compute the distance matrix now:

In [ ]:
D_Matrix_amazRivs_air=AmazonSystem_5880.geometry.apply \
                (lambda river: large_airports.geometry.distance(river)/1000)

In [ ]:
Stat_amz_air=pd.DataFrame()
Stat_amz_air['mean']=D_Matrix_amazRivs_air.mean(axis=1) # mean D to all airports
Stat_amz_air['min']=D_Matrix_amazRivs_air.min(axis=1)# min D to all airports
Stat_amz_air['max']=D_Matrix_amazRivs_air.max(axis=1)# max D to all airports

# see some
Stat_amz_air.head(10)

In [ ]:
# closest river to each airport
D_Matrix_amazRivs_air.idxmin(axis=0).head()

In [ ]:
# farthest river to each airport
D_Matrix_amazRivs_air.idxmax(axis=0).head()

### c. Between Polygon and Point

Let me re use the world rivers to get the rivers in a couple of systems:

In [ ]:
river_systems=world_rivers.query("SYSTEM in ['Amazon','Parana']")
river_systems

Let me combine per system:

In [ ]:
ama_para=river_systems.dissolve(by='SYSTEM')
ama_para.drop(columns='RIVER',inplace=True)
ama_para

We still have lines:

In [ ]:
ama_para.plot(cmap='viridis')

But we will have polygons after this:

In [ ]:
ama_para.convex_hull.plot(cmap='viridis')

As we have a geoseries of two geometries...

In [ ]:
ama_para.convex_hull,type(ama_para.convex_hull)

Let's turn that into a GDF:

In [ ]:
ama_para_hulls=ama_para.convex_hull.to_frame()
ama_para_hulls.rename(columns={0:'geometry'},inplace=True)
ama_para_hulls=ama_para_hulls.set_geometry('geometry')
ama_para_hulls.crs="EPSG:5880"

#voila
ama_para_hulls

And now, the distance matrix:

In [ ]:
D_Matrix_SYSHulls_air=ama_para_hulls.geometry.apply \
                (lambda system: large_airports.geometry.distance(system)/1000)
D_Matrix_SYSHulls_air

From here, you can compute distances between other kinds of geometries.

## II. Clipping

Clipping uses a GDF geometry as a MASK to cut another GDF which suppossedly is bigger and needes to be clipped.

Pay attention to the world rivers again:

In [ ]:
world_rivers

As you see, this GDF has no Country column. But since it has geometry, you can keep the rivers, or their sections, that serve a country:

In [ ]:
rivers_brazil5880 = gpd.clip(gdf=world_rivers.to_crs(5880),
                             mask=brazil5880)

See differences:

- input 1: Rivers

In [ ]:
world_rivers.plot()

- input 2: mask (Brazil)

In [ ]:
brazil5880.plot(facecolor="greenyellow")

> Output: clipped rivers

In [ ]:
rivers_brazil5880.plot()

In [ ]:
base = brazil5880.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
rivers_brazil5880.plot(edgecolor='blue', linewidth=0.5,
                    ax=base)

We can create our own mask for clipping:

Let me get the **bounding box** of the map (the smallest possible rectangle that completely encloses a geometric shape or set of shapes):

In [ ]:
brazil5880.total_bounds #[minx, miny, maxx, maxy]

In [ ]:
# saving the output
minx, miny, maxx, maxy=brazil5880.total_bounds
minx, miny, maxx, maxy

I will combine those coordinates with the _centroid_ (mid_x,mid_y)to create a BOX of the north and south of Brazil:

In [ ]:
north_mask = [minx, mid_y, maxx, maxy]
south_mask = [minx, minx, maxx, mid_y]

# split Brazil
states_brazil5880.clip(north_mask).plot(edgecolor="yellow")

In [ ]:
states_brazil5880.clip(south_mask).plot(edgecolor="yellow")

As you see, with clip we can cut polygons (not respecting the borders).

## III. Spatial Joins

We’re familiar with **merging**, which joins tables using common keys. Spatial joins, by contrast, rely solely on **geometry columns** to perform various types of filtering.

Let me use the brazilian large airports and states:

In [ ]:
large_airports.head()

...and:

In [ ]:
states_brazil5880.head()

In [ ]:
# as before set index for 'states'
states_brazil5880.set_index('state_name',inplace=True)
states_brazil5880.head()

### a. Within

Let's ask:
> The large airports whose geometries are within the borders of a state in Brazil.

In [ ]:
airports_within_states = gpd.sjoin(
    large_airports,         # LEFT: airports we want to filter/keep
    states_brazil5880,      # RIGHT: spatial boundaries to check against
    how='inner',            # return geometries that match in both LEFT/RIGHT (jointype)
    predicate='within'      # spatial condition: LEFT geometry within RIGHT geometry
)

# these are:
airports_within_states

We just performed a point-to-polygon spatial join.
Notice that the result preserves the original geometries from the LEFT GeoDataFrame — specifically, only those features whose spatial relationship satisfied both the predicate (e.g., 'within') and the join type ('inner').
The non-geometric attributes (columns) from the RIGHT GeoDataFrame are joined to the matching rows.

Notice that, the request gave you points. I will add all the states below:

In [ ]:
base=states_brazil5880.explore(edgecolor='black',style_kwds={'fillOpacity': 0.1,'color':'grey'})
airports_within_states.explore(color='red',m=base)

### b. Contains

Importantly, if the LEFT GeoDataFrame geometries are polygons and the RIGHT one points (a polygon-to-point join), you’ll typically need to use a different predicate — such as 'contains' — to express the spatial relationship correctly.

> Brazilian states that house large airports

In [ ]:
states_containing_LargeAirports = gpd.sjoin(states_brazil5880,large_airports,how='inner',
                                            predicate='contains')

states_containing_LargeAirports

As requested, you got polygons (not the airports). I will add airports for reference.

In [ ]:
base=states_containing_LargeAirports.explore(color='red')
large_airports.explore(color='white',m=base)

### c. Intersects

'Contains' and 'within' are literally strict: Any airport located exactly on a state boundary — whether due to data precision, snapping, or real geography — will be excluded, even if it’s “practically” inside the state. More flexibility is achieved with **intersects**.

> A point on the border of Rio de Janeiro, can be detected if I use intersect!

In [ ]:
# this is RDJ:
Rio=states_brazil5880.loc[['Rio de Janeiro'],:]

#this a point on its border
from shapely.geometry import Point

coordinates=Rio.geometry.iloc[0].boundary.geoms[0].coords[1]
aRDJ_PointInBorder=Point(coordinates)
aRDJ_borderPoint = gpd.GeoDataFrame({'name': ['RioJaneiro Border Point']},
                                    geometry=[aRDJ_PointInBorder],
                                    crs=states_brazil5880.crs)

In [ ]:
## Intersects result
gpd.sjoin(aRDJ_borderPoint,Rio,
          how='inner', predicate='intersects')

> A point on the border of Rio de Janeiro, can NOT be detected if I use within / contains

In [ ]:
## within should return no rows
gpd.sjoin(aRDJ_borderPoint,Rio,
          how='inner', predicate='within')

In [ ]:
## contains should return no rows
gpd.sjoin(Rio,aRDJ_borderPoint,
          how='inner', predicate='contains')

We knew this:

In [ ]:
# we need crs4326 for explore:
aRDJ_borderPoint_latlon = aRDJ_borderPoint.to_crs(4326).geometry.iloc[0]

base=Rio.explore(zoom_start=20,location=[aRDJ_borderPoint_latlon.y, aRDJ_borderPoint_latlon.x])
aRDJ_borderPoint.explore(m=base, color='red')

### d. Touches

We also have 'touches', a more stringent predicate than 'intersects'. It returns geometries that:
 - Share a only a border (for polygons or lines), or
 - Have only one tangent point in common.

> Which states are neighbors of 'BAHIA", including BAHIA

In [ ]:
# Neighbors of Bahia?
gpd.sjoin(N_brazil.loc[N_brazil.state_name=='Bahia',:],N_brazil,how='inner', predicate='intersects').shape

That is, Bahia seems to share borders with 5 states:

In [ ]:
base=gpd.sjoin(N_brazil,N_brazil.loc[N_brazil.state_name=='Bahia',:],
               how='inner',
               predicate='intersects').plot(color='yellow',edgecolor='red')
N_brazil.loc[N_brazil.state_name=='Bahia',:].plot(ax=base, color='red')

However, because many free GeoDataFrames — especially those sourced as Shapefiles — contain topological imperfections like gaps, overlaps, or misaligned vertices, 'touches' often fails to detect what should be adjacent features. Ironically, this “failure” can be useful: 'touches' acts as a diagnostic tool — highlighting where boundaries are not perfectly aligned.

In [ ]:
gpd.sjoin(N_brazil.loc[N_brazil.state_name=='Bahia',:],N_brazil,how='inner', predicate='touches').shape

See the neighbor that disappears:

In [ ]:
base=gpd.sjoin(N_brazil,N_brazil.loc[N_brazil.state_name=='Bahia',:],
               how='inner',
               predicate='touches').plot(color='yellow',edgecolor='red')
N_brazil.loc[N_brazil.state_name=='Bahia',:].plot(ax=base, color='red')

### e. Crosses

When we have **lines**, we may need **crosses**. Let me subset our rivers:

In [ ]:
amazonSystem=rivers_brazil5880[rivers_brazil5880.SYSTEM=='Amazon']
amazonSystem.set_index('RIVER',inplace=True)

Then,
> Which rivers from the Amazon system are intersecting states?

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='intersects')

A count of the result:

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='intersects').shape

Alternatively,

> Which rivers from the Amazon system are crossing states?

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='crosses')

You got one less:

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='crosses').shape

Again, _intersects_ means both geometries have some 'space' in common. But **crosses** is an intersection that has to cross the spatial object. From the result above, there is one river that shares space with the state, but is not crossing its border:

In [ ]:
# Get intersects result
intersects_result = gpd.sjoin(amazonSystem,states_brazil5880, how='inner', predicate='intersects')
intersects_result

In [ ]:
# Get crosses result
crosses_result = gpd.sjoin(amazonSystem,states_brazil5880, how='inner', predicate='crosses')
crosses_result

In [ ]:
river_notCrossing=list(set(intersects_result.index)-set(crosses_result.index))
river_notCrossing

In [ ]:
# Find indexes/columns
state_notCrossed=intersects_result.loc[river_notCrossing,'state_name'].to_list()
state_notCrossed

Now we know the river that is not crossing an state, and the name of that state.

In [ ]:
base=states_brazil5880.loc[state_notCrossed,:].plot(color='w',edgecolor='k',figsize=(12, 8))
amazonSystem.plot(ax=base)
amazonSystem.loc[river_notCrossing,:].plot(color='red',ax=base)


## IV. Spatial Overlay

As the name implies, you need two inputs. We might need to create or find some geometries from the geometries we already have. Using a set theory approach, we will see the use of _intersection_, _union_, _difference_, and _symmetric difference_.
Let's remember these results:

In [ ]:
# Create a figure and a 2x2 grid of axes
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
axes = axes.flatten()

N_brazil.plot(color='pink',edgecolor='k',ax=axes[0])
axes[0].set_title("1. North")

S_brazil.plot(color='pink',edgecolor='k',ax=axes[1])
axes[1].set_title("2. South")

W_brazil.plot(color='pink',edgecolor='k',ax=axes[2])
axes[2].set_title("3. West")

E_brazil.plot(color='pink',edgecolor='k',ax=axes[3])
axes[3].set_title("4. East")

plt.show()

Let's play with these GDFs, keep in mind the position of the GDFs:

```
GDFleft.overlay(GDFright, ...)
```

### a. Intersection

We keep what is common between _left_ and _right_ GDFs.

> Intersection of North and South

In [ ]:
NS_brazil=N_brazil.overlay(S_brazil, how="intersection",keep_geom_type=True)
NS_brazil.plot(color='pink',edgecolor='k')

> Intersection of West and East

In [ ]:
WE_brazil=W_brazil.overlay(E_brazil, how="intersection",keep_geom_type=True)
WE_brazil.plot(color='pink',edgecolor='k')

### b. Union

In the union overlay, we do not return dissolved geometries. Instead, we return a complete set of new geometries that represent every unique spatial combination of the two input GeoDataFrames. The resulting GeoDataFrame will contain the columns from both GDFs, and the row count will be the sum of all parts of A, all parts of B, and the newly created intersection parts, potentially inflated by topological flaws


> Unite the West/East intersection with the North/South intersection

In [ ]:
mid_Brazil=NS_brazil.overlay(WE_brazil,how="union",keep_geom_type=True)
mid_Brazil.plot(color='pink',edgecolor='k')

### c. Difference

Here, you keep what belongs to the left GDF that is not in the right GDF.

> Keep nothern states that are not in the southern ones

In [ ]:
NS_diff_brazil=N_brazil.overlay(S_brazil, how='difference')
NS_diff_brazil.plot(color='pink',edgecolor='k')

### d. Symmetric Difference

Here, we keep what is not in the intersection but in both GDFs.

> Unite Northern and Southern states, but keep states that are not in their intersection.

In [ ]:
NS_simdiff_brazil=N_brazil.overlay(S_brazil, how='symmetric_difference')
NS_simdiff_brazil.plot(color='pink',edgecolor='k')


_____________


### e. Cleaning and Overlay

Overlay and SJoin differ in that overlay, as it creates geometries, may not output clean results. Not because overlay fault, but due to the quality of the original maps.


Let me use the **intersection** as our topological detective.

- **Using set operations**: What are the common states between northern and souther states.

In [ ]:
setIntersection=set(N_brazil.state_name) & set(S_brazil.state_name)

# how many, which are they
len(setIntersection),setIntersection

- **The spatial overlay with keep_geom_type as True**:

In [ ]:
NS_brazil_T=N_brazil.overlay(S_brazil, how="intersection",keep_geom_type=True)

# different from 8 (see above)
NS_brazil_T.shape[0]

- **The spatial overlay with keep_geom_type as False**:

In [ ]:
NS_brazil_F=N_brazil.overlay(S_brazil, how="intersection",keep_geom_type=False)

# different from 8 and 11 (see above)
NS_brazil_F.shape[0]

As you see, the intersection that we plotted before had several topological issues we could not see with bare eyes. There were several **additional** geometries that represent flaws. When we used "keep_geom_type=True" the results were polygons (left GDF geometries were polygons); but when we asked "keep_geom_type=False" the result may include polygons and more (even collections). Let's see this last case:

In [ ]:
NS_brazil_F.geometry.geom_type.value_counts()

> Can this be fixed?

Let's see what can be done:

- review the previous intersection:

In [ ]:
NS_brazil_T

- Detect problematic rows

In [ ]:
NS_brazil_T[NS_brazil_T.state_name_1!=NS_brazil_T.state_name_2]

Pernambuco and Minas Gerais must have some of their points "invading" Bahia and  Alagoas polygon. The strategy here is that Bahia and Aalagoas will not push away the invading points, but to 'abandon' the area in conflict (difference) for the sake of clean limits (dissolve).

- We need to work with the original states. Let's make a copy of them before cleaning:

In [ ]:
states_brazil_clean=states_brazil5880.copy()
states_brazil_clean.head()

- Let's dissolve Pernambuco and Minas Gerais, the invaders:

In [ ]:
alienStates=["Minas Gerais","Pernambuco"]
alienUnion_GDF=states_brazil_clean[states_brazil_clean.index.isin(alienStates)].dissolve()
alienUnion_GDF

- Keep the states to be cleaned from the 'alienStates', no dissolving - just filtering:

In [ ]:
forCleaning=["Alagoas","Bahia"] #order matters
forCleaning_GDF=states_brazil_clean[states_brazil_clean.index.isin(forCleaning)]
forCleaning_GDF

- Recreate boundaries, so that no intersection occurs... do you see different values in geometry column now?

In [ ]:
recreatedPolygons=forCleaning_GDF.overlay(alienUnion_GDF,how="difference",keep_geom_type=False).dissolve(by="state_code")
recreatedPolygons

- Replace old geometries with recent values.

This is a sensible moment: You need to change just the geometry cells, not the whole rows:

In [ ]:
#old values
states_brazil_clean.loc[forCleaning,'geometry']

In [ ]:
#new values - notice order
recreatedPolygons.geometry

In [ ]:
# but use this for replacement
recreatedPolygons.geometry.values

Then,

In [ ]:
states_brazil_clean.loc[forCleaning,'geometry']=recreatedPolygons.geometry.values

#see
states_brazil_clean.loc[forCleaning,'geometry']

- We need to recreate northern and southern GDFs, let's redo the four ones at once:

In [ ]:
# the north
N_brazil_clean=states_brazil_clean.reset_index().cx[:,mid_y:]
# the south
S_brazil_clean=states_brazil_clean.reset_index().cx[:,:mid_y]
# the west
W_brazil_clean=states_brazil_clean.reset_index().cx[:mid_x,:]
# the east
E_brazil_clean=states_brazil_clean.reset_index().cx[mid_x:,:]

- Confirm you have improved the intersection with **keep_geom_type=True**:

In [ ]:
N_brazil_clean.overlay(S_brazil_clean, how="intersection",keep_geom_type=True)

- Confirm you have improved the intersection with **keep_geom_type=False**:

In [ ]:
N_brazil_clean.overlay(S_brazil_clean, how="intersection",keep_geom_type=False).geometry.geom_type.value_counts()

All this work did help. But remember while we cleaned the neighborhood issues between 4 states, cleaning 2 of them there might be more cleaning remaining.

**What about** union?

The _mid_Brazil_ had these many geometries (rows):

In [ ]:
len(mid_Brazil)

Just using sets, this is the target count:

In [ ]:
interNS=set(N_brazil_clean.state_name)&set(S_brazil_clean.state_name)
interEW=set(E_brazil_clean.state_name)&set(W_brazil_clean.state_name)
union_interNSEW=interNS|interEW
len(union_interNSEW)

Let's recreate intersection to recreate a clean mid Brazil:

In [ ]:
NS_brazil_clean=N_brazil_clean.overlay(S_brazil_clean, how="intersection",keep_geom_type=True)
EW_brazil_clean=E_brazil_clean.overlay(W_brazil_clean, how="intersection",keep_geom_type=True)

# then
mid_Brazil_clean=NS_brazil_clean.overlay(EW_brazil_clean,how="union",keep_geom_type=True)

# count
len(mid_Brazil_clean)

Again, we had improved, but you realize there is still more cleaning to do.

Finally, let's check the recent result:

In [ ]:
mid_Brazil_clean

The UNION overlay is a simple idea. But the result above does not seem friendly.

Let's recreat this GDF from previous steps:

In [ ]:
NS_brazil_clean

In [ ]:
# better format
NS_brazil_clean.drop(columns=['state_code_1','state_name_2','state_code_2'],inplace=True)
NS_brazil_clean.rename(columns={'state_name_1':'state_name_clean'},inplace=True)
# then
NS_brazil_clean

Notice this one is not cleaned:

In [ ]:
EW_brazil_clean

We will not do the any reformatting as before, since we can not erase columns.

Let's see the 'mid' again:

In [ ]:
mid_Brazil_clean=NS_brazil_clean.overlay(EW_brazil_clean,how="union",keep_geom_type=True)
mid_Brazil_clean

Notice this UNION operation  identifies the intersection in the result. **MATO GROSSO** is present in both.

However, missing values were created where no intersection exists. Remember, whatever is not in the intersection of these two, would be the result of **symmetric difference**:

In [ ]:
NS_brazil_clean.overlay(EW_brazil_clean,how="symmetric_difference",keep_geom_type=True)

As you see, the fact that we have not clean all the geometries affects all these overlay operations.

**NEXT STEPS**

- Cleaning spatial data is not trivial. It is time consuming. We could keep working here to improve this.
- We may need to take this fight to QGis or ArcGis to edit the points manually, it gives you more control, but it will affect replicability.
- A smart decision would be to get a new map  with a [better quality](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/) - which might also require some money investment.